## V_MRDH model: OD demand for different modalities

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### OD matrices

In [ ]:
modalities = ["Auto", "Fiets", "OV"]
periods = ["os", "rd", "as"]  # ochtendspits (7-9), restdag, avondspits (16-18)

od = {}

for modality in modalities:
    for period in periods:
        od[modality, period] = pd.read_excel('V_MRDH_MatComp_2020.xlsx', sheet_name=f"{modality}_{period}", index_col=0, usecols='A:BN', nrows=65)
                                 
od_etm = {}
for modality in modalities:
    od_etm[modality] = pd.read_excel('V_MRDH_MatComp_2020.xlsx', sheet_name=f"{modality}_etm", index_col=0, usecols='A:BN', nrows=65)

In [ ]:
od_etm["Auto"].head(5)

In [ ]:
# Add the number for each column and index
for modality in modalities:
    for period in periods:
        od[modality, period].columns = range(1, 66)
        od[modality, period].index = range(1, 66)
    od_etm[modality].columns = range(1, 66)
    od_etm[modality].index = range(1, 66)

In [ ]:
# Calculate the total demand for each period
od_totaal = {}
for period in periods:
    od_totaal[period] = sum(od[modality, period] for modality in modalities)

od_totaal_etm = sum(od_etm[modality] for modality in modalities)

### Inbound / outbound traffic

In [ ]:
# Calculate the total inbound and outbound traffic for area 1 to 65 in each period
inbound = {}
outbound = {}
diff = {}
for period in periods:
    inbound[period] = od_totaal[period].sum(axis=0)
    outbound[period] = od_totaal[period].sum(axis=1)
    diff[period] = inbound[period] - outbound[period]

inbound_df = pd.DataFrame(inbound)
outbound_df = pd.DataFrame(outbound)
total_df = inbound_df + outbound_df
diff_df = pd.DataFrame(diff)
ratio_df = diff_df.div(total_df, axis=0)

# Plot heatmaps of the diff df
fig, ax = plt.subplots(3, 1, figsize=(16, 4.5))
# Use another color map for total_df, starting at white: 
sns.heatmap(total_df.iloc[:50, :].T, cmap="Greens", square=True, ax=ax[0])
ax[0].set_title("Total traffic for each period")
sns.heatmap(diff_df.iloc[:50, :].T, cmap='coolwarm', center=0, square=True, ax=ax[1])
# Add a title
ax[1].set_title("Difference between inbound and outbound traffic for each period")
sns.heatmap(ratio_df.iloc[:50, :].T, cmap='coolwarm', center=0, square=True, ax=ax[2])
ax[2].set_title("Ratio between inbound and outbound traffic for each period (negative means more outbound)")
# Set the y-axis labels Ochtendspits, Restdag, Avondspits
for x in ax:
    x.set_yticklabels(["Ochtendspits (7-9)", "Restdag", "Avondspits (16-18)"], rotation=0)

plt.tight_layout()
plt.savefig(f"../img/inbound_outbound_traffic.png", dpi=300, bbox_inches='tight')

In [ ]:
# Normalize over the row, so that the sum of each row is 1
od_totaal_norm = {}
for modality, df in od_totaal.items():
    od_totaal_norm[modality] = df.div(df.sum(axis=1), axis=0)
    
# Save as NumPy arrays with uint16 datatype to save memory
# TODO

### External car traffic

In [ ]:
# TODO
# Create a numpy array with the car traffic from the external areas to the city areas and visa versa
int_mrhd65_areas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 28, 29, 31, 34, 41, 43, 44, 45]
ext_mrdh65_areas = [14, 32, 35, 37, 38, 39, 40, 53, 54, 55, 56, 57, 58]

In [ ]:
# Area numbers are from:
#     # UXsim world (from traffic.py)
#     self.uw = get_uxsim_world(save_mode=False, show_mode=True)
#     # mrdh65 areas without population but covered in the network
#     self.mrdh65s_ext = list(set(self.uw.node_mrdh65_dict.keys()) - set(self.mrdh65s))
#     self.mrdh65s_ext.remove(0)
#     print(f"MRDH65 areas: {self.mrdh65s} (total {len(self.mrdh65s)}), extended areas: {self.mrdh65s_ext} (total {len(self.mrdh65s_ext)})")

In [ ]:
# Plot the internal and external areas
fig, ax = plt.subplots(figsize=(8, 8))
areas_65.loc[int_mrhd65_areas].plot(ax=ax, color='red', alpha=0.5)
areas_65.loc[ext_mrdh65_areas].plot(ax=ax, color='blue', alpha=0.5)
ax.set_title("Areas with simulated population (red) and without simulated population (blue)")
plt.savefig(f"../img/internal_external_areas.png", dpi=300, bbox_inches='tight')

In [ ]:
# Create a selection of od. Use uint16 datatype to save memory
od_ext_into_city = od["Auto"].loc[ext_mrdh65_areas, int_mrhd65_areas].astype('uint16')
od_ext_out_city = od["Auto"].loc[int_mrhd65_areas, ext_mrdh65_areas].astype('uint16')

# Save both as pickle
od_ext_into_city.to_pickle('../data/od_ext_into_city.pkl')
od_ext_out_city.to_pickle('../data/od_ext_out_city.pkl')